In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

2023-11-09 04:45:51,688 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from pyspark.sql.functions import *

In [4]:
data_df=spark.read.format("json").load("file:/home/labuser/Downloads/688b97cc-73ed-4262-b3be-6ef274e26311_83d04ac6-cb74-4a96-a06a-e0d5442aa126_cars.json")
data_df.show()

+--------+----------+--------------+--------+---------+
|    kind|      make|         model|   price|    style|
+--------+----------+--------------+--------+---------+
|electric|     Telsa|       Model S| 71000.0|    sedan|
|  hybrid|      Audi|     A3 E-Tron| 37900.0|   luxury|
|  hybrid|       BMW|          330e| 43700.0|    sedan|
|electric|       BMW|            i3| 43300.0|    sedan|
|  hybrid|       BMW|            i8|137000.0|    coupe|
|  hybrid|       BMW|  X5 xdrive40e| 64000.0|      suv|
|electric|     Chevy|      Spark EV| 26000.0|    coupe|
|electric|     Chevy|          Volt| 34000.0|    sedan|
|electric|      Fiat|          500e| 32600.0|    coupe|
|  hybrid|      Ford|  C-Max Energi| 32600.0|wagon/van|
|electric|      Ford|Focus Electric| 29200.0|    sedan|
|electric|      Ford| Fusion Energi| 33900.0|    sedan|
|  hybrid|   Hyundai|        Sonata| 35400.0|    sedan|
|electric|       Kia|       Soul EV| 34500.0|    sedan|
|electric|  Mercedes|       B-Class| 42400.0|   

In [5]:
data_df.select("make").distinct().count()

15

In [6]:
data_df.select("price").filter("price > 60000").count()

9

In [9]:
b1=data_df.withColumn("ispremium",when(col("price")>60000,'1').otherwise("0")).select("style","kind","make","ispremium")
b1.orderBy(col("style").desc()).show()

+---------+--------+----------+---------+
|    style|    kind|      make|ispremium|
+---------+--------+----------+---------+
|wagon/van|  hybrid|      Ford|        0|
|      suv|  hybrid|     Volvo|        1|
|      suv|  hybrid|       BMW|        1|
|      suv|  hybrid|  Mercedes|        1|
|      suv|  hybrid|   Porsche|        1|
|      suv|electric|     Tesla|        1|
|    sedan|electric|     Telsa|        1|
|    sedan|electric|      Ford|        0|
|    sedan|electric|       BMW|        0|
|    sedan|  hybrid|       BMW|        0|
|    sedan|  hybrid|   Hyundai|        0|
|    sedan|electric|       Kia|        0|
|    sedan|  hybrid|  Mercedes|        0|
|    sedan|electric|    Nissan|        0|
|    sedan|electric|     Chevy|        0|
|    sedan|electric|      Ford|        0|
|    sedan|electric|  Mercedes|        0|
|    sedan|electric|Mitsubishi|        0|
|    sedan|  hybrid|   Porsche|        1|
|    sedan|electric|     Tesla|        0|
+---------+--------+----------+---

In [12]:
data_df.groupBy("style").agg(avg("price")).show()

+---------+-----------------+
|    style|       avg(price)|
+---------+-----------------+
|   luxury|          37900.0|
|    coupe|          67900.0|
|      suv|          71600.0|
|    sedan|42471.42857142857|
|wagon/van|          32600.0|
+---------+-----------------+



In [39]:


data_df=spark.read.option("multiline","true").format("json").load("file:/home/labuser/Downloads/bb886084-3fc4-467d-b6c7-f55ae81b4f75_83d04ac6-cb74-4a96-a06a-e0d5442aa126_client.json")
data_df.printSchema()
data_df.show()

root
 |-- age: long (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- familyName: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

+---+-----------+-----------+---+---------+
|age|countryCode| familyName| id|     name|
+---+-----------+-----------+---+---------+
| 33|         US|familyName1|  1|testName1|
| 43|         DE|familyName2|  2|testName2|
| 53|         US|familyName3|  3|testName3|
| 63|         CH|familyName4|  4|testName4|
| 73|         US|familyName5|  5|testName5|
| 23|         DE|familyName6|  6|testName6|
| 36|         US|familyName7|  7|testName7|
| 38|         CH|familyName8|  8|testName8|
+---+-----------+-----------+---+---------+



In [33]:
result_df = data_df.groupBy("countryCode") \
    .agg(
        avg("age").alias("The Average age of Client"),
        max(col("age")).alias("The Max age of Client"),
        min(col("age")).alias("The Min age of Client"),
        percentile_approx(col("age"), 0.5).alias("The Median age of Client"),
        count(col("age").alias("Total No of clients"))
    ) \
    .orderBy(col("countryCode").desc())

result_df.show()



+-----------+-------------------------+---------------------+---------------------+------------------------+-----------------------------------+
|countryCode|The Average age of Client|The Max age of Client|The Min age of Client|The Median age of Client|count(age AS `Total No of clients`)|
+-----------+-------------------------+---------------------+---------------------+------------------------+-----------------------------------+
|         US|                    48.75|                   73|                   33|                      36|                                  4|
|         DE|                     33.0|                   43|                   23|                      23|                                  2|
|         CH|                     50.5|                   63|                   38|                      38|                                  2|
+-----------+-------------------------+---------------------+---------------------+------------------------+----------------------

In [4]:
data_df=spark.read.format("json").load("file:/home/labuser/Downloads/5d81b319-291a-41be-afe4-ab8d39e283b8_83d04ac6-cb74-4a96-a06a-e0d5442aa126_companies.json")
data_df.printSchema()
data_df.show()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- acquisition: struct (nullable = true)
 |    |-- acquired_day: long (nullable = true)
 |    |-- acquired_month: long (nullable = true)
 |    |-- acquired_year: long (nullable = true)
 |    |-- acquiring_company: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- permalink: string (nullable = true)
 |    |-- price_amount: long (nullable = true)
 |    |-- price_currency_code: string (nullable = true)
 |    |-- source_description: string (nullable = true)
 |    |-- source_url: string (nullable = true)
 |    |-- term_code: string (nullable = true)
 |-- acquisitions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- acquired_day: long (nullable = true)
 |    |    |-- acquired_month: long (nullable = true)
 |    |    |-- acquired_year: long (nullable = true)
 |    |    |-- company: struct (nullable = true)
 |    |    |    |-- name: strin

2023-11-09 04:47:40,401 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------------+--------------+---------------+--------------------+-------------------+--------------------+-----------+-------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+----------------+--------------------+--------------------+
|                 _id|         acquisition|        acquisitions|        alias_list|       blog_feed_url|            blog_url|  category_code|        competitions|          created_at|      crunc

In [7]:
data_df.select("name").distinct().show()

+----------------+
|            name|
+----------------+
|SomethingSimpler|
|     BlueLithium|
|          Helium|
|     Adknowledge|
|           Zmags|
|        Phanfare|
|         octabox|
|          Ninite|
|    CreateDebate|
| PrimeTimeRewind|
|          itzbig|
|         NotePub|
|           eVapt|
|       Blue Nile|
|       FIRST ROI|
|       Telelogic|
|           Tyroo|
|     Prospection|
|           POPrl|
|     Bertelsmann|
+----------------+
only showing top 20 rows



In [9]:
data_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- acquisition: struct (nullable = true)
 |    |-- acquired_day: long (nullable = true)
 |    |-- acquired_month: long (nullable = true)
 |    |-- acquired_year: long (nullable = true)
 |    |-- acquiring_company: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- permalink: string (nullable = true)
 |    |-- price_amount: long (nullable = true)
 |    |-- price_currency_code: string (nullable = true)
 |    |-- source_description: string (nullable = true)
 |    |-- source_url: string (nullable = true)
 |    |-- term_code: string (nullable = true)
 |-- acquisitions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- acquired_day: long (nullable = true)
 |    |    |-- acquired_month: long (nullable = true)
 |    |    |-- acquired_year: long (nullable = true)
 |    |    |-- company: struct (nullable = true)
 |    |    |    |-- name: strin

In [8]:
data=data_df.withColumn("base1",explode(col("funding_rounds"))).select("base1")\
            .withColumn("base2",explode(col("base1.investments.financial_org.name"))).select("base2").distinct().show()

+--------------------+
|               base2|
+--------------------+
|Nordic Venture Pa...|
|Virginia's Center...|
| Oak Hill Investment|
|     iSherpa Capital|
|   Sycamore Ventures|
|   RBC Dain Rauscher|
|       Iris Ventures|
|     Satwik Ventures|
|                 TNP|
|Miven Venture Par...|
|Yasuda Enterprise...|
|U.S Venture Partners|
|          21Ventures|
|        SSM Partners|
|Axis Capital Corp...|
|  The Zitelman Group|
|Devon Park BioVen...|
|       CDIB Ventures|
| Silicon Valley Bank|
|        Milk Capital|
+--------------------+
only showing top 20 rows



In [10]:
data=data_df.withColumn("base1",explode(col("competitions"))).select("base1")\
            .withColumn("name",col("base1.competitor.name"))\
            .withColumn("permalink",col("base1.competitor.permalink")).select("name","permalink").show()

+--------------------+-------------+
|                name|    permalink|
+--------------------+-------------+
|               Wikia|        wikia|
|             JotSpot|      jotspot|
|          Socialtext|   socialtext|
|  Ning by Glam Media|         ning|
|              Soceeo|       soceeo|
|                Yola|         yola|
|            SocialGO|     socialgo|
|            IslamNor|     islamnor|
|            Empressr|     empressr|
|Transmedia Corpor...|   transmedia|
|      Live Documents|livedocuments|
|            LongJump|     longjump|
|              Phuser|       phuser|
|            NetSuite|     netsuite|
|              Google|       google|
|            Flypaper|     flypaper|
|          MovingLabs|   movinglabs|
|               Ephox|        ephox|
|             Coghead|      coghead|
|         SlideRocket|  sliderocket|
+--------------------+-------------+
only showing top 20 rows



In [11]:
data=data_df.withColumn("base1",explode(col("funding_rounds"))).select("base1")\
            .withColumn("raised_amount",col("base1.raised_amount").cast("int"))\
            .withColumn("companies",col("base1.investments.financial_org.name")).select("raised_amount","companies").show(truncate=False)

+-------------+------------------------------------------------------------------------------------+
|raised_amount|companies                                                                           |
+-------------+------------------------------------------------------------------------------------+
|5250000      |[Frazier Technology Ventures, Trinity Ventures]                                     |
|9500000      |[Accel Partners, Frazier Technology Ventures, Trinity Ventures]                     |
|25000000     |[DAG Ventures, Accel Partners, Trinity Ventures, Frazier Technology Ventures]       |
|8500000      |[Greylock Partners, Omidyar Network]                                                |
|2800000      |[Greylock Partners, Omidyar Network, null, null, null, null, FLOODGATE, SV Angel]   |
|28700000     |[Highland Capital Partners, Greylock Partners, Omidyar Network, SVB Financial Group]|
|5000000      |[]                                                                          

In [66]:
data_df=spark.read.format("json").load("file:/home/labuser/Downloads/688b97cc-73ed-4262-b3be-6ef274e26311_83d04ac6-cb74-4a96-a06a-e0d5442aa126_cars.json")
data_df.show()


+--------+----------+--------------+--------+---------+
|    kind|      make|         model|   price|    style|
+--------+----------+--------------+--------+---------+
|electric|     Telsa|       Model S| 71000.0|    sedan|
|  hybrid|      Audi|     A3 E-Tron| 37900.0|   luxury|
|  hybrid|       BMW|          330e| 43700.0|    sedan|
|electric|       BMW|            i3| 43300.0|    sedan|
|  hybrid|       BMW|            i8|137000.0|    coupe|
|  hybrid|       BMW|  X5 xdrive40e| 64000.0|      suv|
|electric|     Chevy|      Spark EV| 26000.0|    coupe|
|electric|     Chevy|          Volt| 34000.0|    sedan|
|electric|      Fiat|          500e| 32600.0|    coupe|
|  hybrid|      Ford|  C-Max Energi| 32600.0|wagon/van|
|electric|      Ford|Focus Electric| 29200.0|    sedan|
|electric|      Ford| Fusion Energi| 33900.0|    sedan|
|  hybrid|   Hyundai|        Sonata| 35400.0|    sedan|
|electric|       Kia|       Soul EV| 34500.0|    sedan|
|electric|  Mercedes|       B-Class| 42400.0|   

In [76]:
data_df.repartition(1).write.option("header",True) \
        .partitionBy("make") \
        .mode("overwrite") \
        .csv("file:/home/labuser/Downloads/new_data")